# Ejercicio: construir gráficos interactivos

En este ejercicio vamos a visualizar los datos de demanda y oferta de consultas de especialidad médica y reproducir los [análisis de brechas en el SSMSO 2021](https://www.revistamedicadechile.cl/index.php/rmedica/article/view/10435). Para ello necesitamos:

1. Cargar el cubo de datos
2. Hacer un análisis exploratorio de datos
  - Crear una tabla de contingencia (oferta vs demanda por establecimiento y año)
3. Crear una función para graficar las series de tiempo
  - Filtrar por especialidad
  - Estratificar por establecimiento
4. Graficar las brechas de consultas

# Configurar área de trabajo y cargar datos
Usaremos las librerías de [Plotly](https://plotly.com/graphing-libraries/) para construir gráficos interactivos 📊✨.

In [ ]:
# importar librerías
import pandas as pd
import os
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# definir carpetas de trabajo
datos_multidimensionales = 'https://github.com/rlagosb/taller_eiv/raw/refs/heads/main/datos_multidimensionales/'

In [ ]:
# cargar datos
cubo = pd.read_excel(datos_multidimensionales + 'Cubo_consultas_nuevas.xlsx')

## 🏁 Discusión

1. ¿Qué métricas y dimensiones tiene esta tabla?
2. ¿Cuántas observaciones por año contiene?

In [ ]:
cubo.info()

## 🍎 Desafío

Cree una tabla de contingencia para reportar la oferta y demanda por año y especialidad

# Graficar series de tiempo
Construyamos un gráfico de la serie de tiempo con interactividad básica

In [ ]:
# Veamos que necesitamos procesar los datos para visualizarlos adecuadamente

# prompt: genera un gráfico de Oferta_consultas por periodo desglosado por Centro_siglas

# Generar el gráfico
fig = px.line(cubo,
              x="Periodo",
              y="Oferta_consultas",
              color="Centro_siglas",
              title="Oferta de Consultas por Periodo y Centro")

fig.show()

## 🏁 Discusión
¿Porqué se ve así la serie de tiempo *Oferta_consultas*?

In [ ]:
serie = cubo.groupby(['Periodo','Centro_siglas']).agg({'Oferta_consultas':'sum'}).reset_index()

# Generar el gráfico
fig = px.line(serie,
              x="Periodo",
              y="Oferta_consultas",
              color="Centro_siglas",
              title="Oferta de Consultas por Periodo y Centro")

fig.show()

## Gráfico interactivo

Para explorar las series, en vez de generar un script y un extracto para visualizar cada serie, podemos crear una función que reciba como parámetro la serie que queremos visualizar.

In [ ]:
# Generamos una función que toma como parámetro el nombre de la serie:
# 'Consultas_producidas', 'Consultas_inasistencia', 'Oferta_consultas', 'Consultas_solicitadas', 'Lista_espera_inicial'

def graficar_serie(serie):

  df = cubo.groupby(['Periodo','Centro_siglas']).agg({serie:'sum'}).reset_index()

  # Generar el gráfico
  fig = px.line(df,
                x="Periodo",
                y=serie,
                color="Centro_siglas",
                title=serie+'  por Periodo y Centro')
  fig.show()


In [ ]:
# Ejecutamos la función

graficar_serie('Consultas_producidas')

In [ ]:
# También podemos agregar un parámetro que permita filtrar por especialidad

def graficar_serie(serie, especialidad):

  # generamos una variable para filtrar por especialidad
  filtro_especialidad = (cubo['Especialidad']==especialidad)
  df = cubo[filtro_especialidad].copy()

  df = cubo.groupby(['Periodo','Centro_siglas']).agg({serie:'sum'}).reset_index()
  if serie == 'Lista_espera_inicial': df[serie] = df[serie]/3     # métrica no es aditiva al agregar por trimestre (3 meses)


  # Generar el gráfico
  titulo = serie+'  por Periodo y Centro ('+especialidad+')'
  fig = px.line(df,
                x="Periodo",
                y=serie,
                color="Centro_siglas",
                title=titulo)
  fig.show()


In [ ]:
# Ejecutamos la función para un caso

graficar_serie('Oferta_consultas', 'DERMATOLOGIA')

## 🍎 Desafíos

1. Explore otras series de tiempo cambiando los parámetros serie y/o especialidad. Ejecutando `cubo.Especialidad.unique()` puede obtener el listado de las especialidades
2. Agregue un parámetro a la función *graficar_serie* que permita filtrar los datos para un año específico

## Desglosar por establecimiento

In [ ]:
# prompt: modify 'graficar_series' so that each chart is next to each other sharing y axis

def graficar_serie_especialidad(serie, especialidad, anio):

  # generamos una variable para filtrar por especialidad y otra para el año
  filtro_especialidad = (cubo['Especialidad']==especialidad)
  filtro_año = (cubo['Año']==anio)
  df = cubo[filtro_especialidad & filtro_año].copy()

  # Obtenemos el listado de centros en 'Centro_siglas'
  centros = df['Centro_siglas'].unique()

  # Generamos una grilla de gráficos
  fig = make_subplots(rows=1, cols=len(centros), shared_yaxes=True,
                      subplot_titles=[f"{centro}" for centro in centros])

  # Generar serie por hospital y agregarlo a la grilla
  for i, centro in enumerate(centros):
    df_centro = df[df['Centro_siglas'] == centro].copy()

    # Generamos la serie
    df_centro = df_centro.groupby(['Periodo']).agg({serie:'sum'}).reset_index()
    if serie == 'Lista_espera_inicial': df_centro[serie] = df_centro[serie]/3

    fig.add_trace(go.Scatter(x=df_centro['Periodo'], y=df_centro[serie],
                             mode='lines+markers', name=centro),
                  row=1, col=i+1)

  fig.update_layout(title_text=f"{serie} por Periodo ({especialidad} {anio})",
                    yaxis_range=[0, None])
  fig.show()

In [ ]:
graficar_serie_especialidad('Consultas_solicitadas', 'DERMATOLOGIA', 2021)

## 🏁 Discusión

1. ¿Qué ventajas y desventajas tiene generar las visualizaciones con un lenguaje de programación vs. realizarlos con Excel?
2. ¿Qué ventajas y desventajas tiene analizar los datos utilizando funciones como *graficar_series*?

# Graficar brechas

In [ ]:
def graficar_brechas(especialidad, anio):

  # generamos variables para filtrar por especialidad y año
  filtro_especialidad = (cubo['Especialidad']==especialidad)
  filtro_anio = (cubo['Año']==anio)
  df = cubo[filtro_especialidad & filtro_anio].copy()

  df = df.groupby(['Periodo']).agg({'Oferta_consultas':'sum',
                                    'Consultas_solicitadas':'sum',
                                    'Lista_espera_inicial':'sum'}).reset_index()
  df['Lista_espera_inicial'] = df['Lista_espera_inicial']/3

  # Agregamos la Demanda y Oferta acumulada
  df['Demanda_acumulada'] = df['Consultas_solicitadas'].cumsum()
  df['Oferta_acumulada'] = df['Oferta_consultas'].cumsum()

  # Generar el gráfico
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=df['Periodo'], y=df['Lista_espera_inicial'],
                         fill='tozeroy', name='Lista_espera_inicial',stackgroup='one'))
  fig.add_trace(go.Scatter(x=df['Periodo'], y=df['Demanda_acumulada'],
                         fill='tozeroy', name='Demanda_acumulada',stackgroup='one'))
  fig.add_trace(go.Scatter(x=df['Periodo'], y=df['Oferta_acumulada'],
                         mode='lines+markers', name='Oferta_acumulada'))
  fig.update_layout(title_text=f"Brechas por Periodo ({especialidad} {anio})")
  fig.show()

In [ ]:
graficar_brechas('PSIQUIATRIA ADULTO',2021)

Finalmente generamos una función para desglosar las brechas por establecimiento

In [ ]:
def graficar_brechas_centros(especialidad, anio):

  # generamos una variable para filtrar por especialidad y otra para el año
  filtro_especialidad = (cubo['Especialidad']==especialidad)
  filtro_anio = (cubo['Año']==anio)
  df = cubo[filtro_especialidad & filtro_anio].copy()

  df = df.groupby(['Trimestre','Centro_siglas']).agg({'Oferta_consultas':'sum',
                                    'Consultas_solicitadas':'sum',
                                    'Lista_espera_inicial':'first'}).reset_index()

  # Obtenemos el listado de centros en 'Centro_siglas'
  centros = df['Centro_siglas'].unique()

  # Generamos una grilla de gráficos
  fig = make_subplots(rows=1, cols=len(centros), shared_yaxes=True,
                      subplot_titles=[f"{centro}" for centro in centros])

  # Generar gráfico por hospital y agregarlo a la grilla
  for i, centro in enumerate(centros):

      # Calculamos la Demanda y Oferta acumulada
      df_centro = df[df['Centro_siglas'] == centro].copy()
      df_centro.loc[:,'Demanda_acumulada'] = df_centro['Consultas_solicitadas'].cumsum()
      df_centro.loc[:,'Oferta_acumulada'] = df_centro['Oferta_consultas'].cumsum()

      # Generamos los gráficos
      fig.add_trace(go.Scatter(x=df_centro['Trimestre'], y=df_centro['Oferta_acumulada'],
                             mode='lines+markers', name='Oferta_acumulada',
                               line_color='red',showlegend=(i==0)),
                  row=1, col=i+1)
      fig.add_trace(go.Scatter(x=df_centro['Trimestre'], y=df_centro['Lista_espera_inicial'],
                             fill='tozeroy', name='Lista_espera_inicial',
                               stackgroup='one',line_color='darkviolet',
                               showlegend=(i==0)),
                  row=1, col=i+1)
      fig.add_trace(go.Scatter(x=df_centro['Trimestre'], y=df_centro['Demanda_acumulada'],
                             fill='tozeroy', name='Demanda_acumulada',
                               stackgroup='one',line_color='violet',
                               showlegend=(i==0)),
                  row=1, col=i+1)

  # Agregamos título y publicamos
  fig.update_layout(title_text=f"Brechas por Periodo ({especialidad} {anio})")
  fig.show()


In [ ]:
# Construimos el gráfico para una especialidad

graficar_brechas_centros('PSIQUIATRIA ADULTO',2021)

## 🍎 Desafíos
1. Ejecute *generar_brechas_centros* para otra especialidad
  - Ejecutando `cubo.Especialidad.unique()` puede obtener el listado de las especialidades
1. Modifique alguna de las visualizaciones incorporando otras variables disponibles en el cubo.
2. Genere otra visualización de las brechas que le parezca interesante para contrastar la demanda y oferta.

## 🏁 Discusión

1. ¿Qué tendría que hacer para reproducir el análisis con datos de 2024?
4. ¿Qué tendría que hacer para replicar el análisis en otro Servicio de Salud?
4. ¿Qué tendría que hacer si necesitara incorporar variables que no están en el cubo (ejemplo: solicitudes de interconsultas GES)?